In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

Initialize an empty list to store data for different thresholds

In [ ]:
thresholds = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
data_list = []

In [ ]:
for threshold in thresholds:
    column_name = f'result_thr_{int(threshold * 100)}'

    # Apply the threshold to create a new column in the original DataFrame
    aug_pred_prob_byrd_70_epic[column_name] = aug_pred_prob_byrd_70_epic.apply(lambda row: row['predicted_class'] if row['max_prob'] > threshold else 0, axis=1)
    
    # Filter the DataFrame based on the new column
    test_threshold = aug_pred_prob_byrd_70_epic[aug_pred_prob_byrd_70_epic[column_name] != 0]
    
    # Calculate classification report and confusion matrix
    report = classification_report(test_threshold['EANDM_Final_code_1'], test_threshold[column_name], output_dict=True)
    conf_matrix = confusion_matrix(test_threshold['EANDM_Final_code_1'], test_threshold[column_name])
    
    # Create a data dictionary for this threshold
    data = {
        'Threshold': [threshold],
        'Accuracy': [report['accuracy']],
        'Coverage': [len(test_threshold) / len(aug_pred_prob_byrd_70_epic)],
        'Total': [len(aug_pred_prob_byrd_70_epic)]
    }

        # Add 'macro avg' and 'weighted avg' to the data dictionary
    data['Support'] = [report['macro avg']['support']]
    data['macro avg_F1_Score'] = [report['macro avg']['f1-score']]
    data['weighted avg_F1_Score'] = [report['weighted avg']['f1-score']]
    
    # Extract precision, recall, f1-score, and support for each class
    for class_name in report.keys():
        if class_name not in ['accuracy', 'weighted avg', 'macro avg']:
            data[class_name + '_Precision'] = [report[class_name]['precision']]
            data[class_name + '_Recall'] = [report[class_name]['recall']]
            data[class_name + '_F1_Score'] = [report[class_name]['f1-score']]
            data[class_name + '_Support'] = [report[class_name]['support']]
    
    # Add 'macro avg' and 'weighted avg' to the data dictionary
    # data['macro avg_F1_Score'] = [report['macro avg']['f1-score']]
    # data['weighted avg_F1_Score'] = [report['weighted avg']['f1-score']]
    # data['Support'] = [report['macro avg']['support']]
    
    
    # Append the data for this threshold to the list
    data_list.append(data)

Create the final DataFrame by concatenating the data for different thresholds

In [ ]:
df = pd.concat([pd.DataFrame(data) for data in data_list], ignore_index=True)

Display the final DataFrame

In [ ]:
df